In [ ]:
import numpy as np
import tensorflow as tf
from PIL import Image
import matplotlib.pyplot as plt
import random
import os
# just for TF to see a single GPU in the server
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

# min-max steps the network will be required to learn 
N = (300, 400)
# number fo parallel generation since it's stochastic 
BATCH_SIZE = 6
# n. of channels (RGBa + N others for information transmission)
OUT_DIMS = 32
# trainin epochs
EPOCHS = 10000

In [2]:
# path of the image
image_path = "symbols/emojismall.png"
# import the image
image = np.array(Image.open(image_path), dtype = float)[None, ...]
# normalize the imported image
image = image/255.

In [4]:
class CA (tf.keras.Model):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

        # define Sobel filters and prepare for depthwise convolution 
        self.sobel_x = tf.convert_to_tensor([
            [-1, 0, 1],
            [-2, 0, 2],
            [-1, 0, 1]
        ], dtype = tf.float32)[..., None]
        self.sobel_y = tf.convert_to_tensor([
            [-1, -2, -1],
            [0, 0, 0],
            [1, 2, 1]
        ], dtype = tf.float32)[..., None]

        self.sobel_y = tf.repeat(self.sobel_y, OUT_DIMS, axis = -1)[..., None]
        self.sobel_x = tf.repeat(self.sobel_x, OUT_DIMS, axis = -1)[..., None]

        # layers of the network
        self.batch_norm1 = tf.keras.layers.BatchNormalization()
        #self.batch_norm2 = tf.keras.layers.BatchNormalization()
        self.batch_norm3 = tf.keras.layers.BatchNormalization()
        self.convlayer1 = tf.keras.layers.Conv2D(128, 1, padding = "SAME", activation = tf.nn.leaky_relu)
        #self.convlayer2 = tf.keras.layers.Conv2D(128, 1, padding = "SAME", activation = tf.nn.leaky_relu)
        self.convlayer3 = tf.keras.layers.Conv2D(OUT_DIMS, 1, padding = "SAME", activation = "linear",
                                                kernel_initializer=tf.initializers.zeros(),
                                                bias_initializer=tf.initializers.zeros())
        self.maxpool = tf.keras.layers.MaxPooling2D((3,3), 1, padding= "SAME")

    def call(self, inputs):
        # apply Sobel filters
        grad_x = tf.nn.depthwise_conv2d(inputs, self.sobel_x, strides = [1, 1, 1, 1], padding = "SAME")
        grad_y = tf.nn.depthwise_conv2d(inputs, self.sobel_y, strides = [1, 1, 1, 1], padding = "SAME")

        # concatenate to create the information of each cell
        result = tf.concat((inputs, grad_x, grad_y), axis=-1)

        # forward pass of the ffnn-per-pixel
        result = self.batch_norm1(result)
        result = self.convlayer1(result)
        #result = self.batch_norm2(result)
        #result = self.convlayer2(result)
        result = self.batch_norm3(result)
        result = self.convlayer3(result)

        # randomly remove 20% of the pixels
        rand_mask = tf.random.uniform(tf.shape(inputs)[0:3].numpy().tolist() + [1]) < 0.8
        rand_mask = tf.cast(rand_mask, dtype = tf.float32)
        rand_mask = tf.repeat(rand_mask, tf.shape(inputs)[-1], axis=-1)

        result = result * rand_mask
        result = result + inputs

        # 0-out the pixels that are predicted to be dead 
        # TODO: gradient won't flow, maybe better to use estimators or reinforcement learning
        alive = self.maxpool(result[:,:,:,3:4]) > 0.1
        alive = tf.cast(alive, dtype = tf.float32)
        alive = tf.repeat(alive, tf.shape(inputs)[-1], axis=-1)

        result = result * alive

        return result

In [5]:
# create the initial image and set the central pixel to 1
empty_image = np.zeros((BATCH_SIZE, )+ image.shape[1:3]+(OUT_DIMS,))
empty_image[:, empty_image.shape[1]//2, empty_image.shape[2]//2, 3:]=1

In [8]:
model = CA()

In [ ]:
optimizer = tf.optimizers.legacy.Adam(learning_rate=1e-4)

# training loop, store the model every 10 epochs
for i in range(EPOCHS):
    with tf.GradientTape() as tape:
        current_image = np.copy(empty_image)
        for j in range(random.randint(*N)):
            current_image = model(current_image)
        current_image = current_image[:, :, :, 0:4]

        loss = tf.reduce_mean((current_image-image)**2)
    grad = tape.gradient(loss,model.trainable_weights)
    optimizer.apply_gradients(zip(grad, model.trainable_weights))
    
    if (i+1) % 10 == 0:
        print(f" epoch {i} - loss {loss}")
        model.save_weights(f"./checkpoints/{i}/checkpoint""")
        model.save(f"./checkpoints/{i}/my_model.keras")
    else:
        print("*", end="")

In [11]:
m_ghost = CA()
m_ghost.load_weights("checkpoints-ghost/final/checkpoint")

m_emoji = CA()
m_emoji.load_weights("checkpoints-emoji/final/checkpoint")

In [ ]:
# create images of the generation 
from tqdm import trange
c_ghost = np.zeros((1, )+ (64,64)+(OUT_DIMS,))
c_ghost[:, c_ghost.shape[1]//2, c_ghost.shape[2]//2, 3:]=1
c_emoji = np.zeros((1, )+ (40,40)+(OUT_DIMS,))
c_emoji[:, c_emoji.shape[1]//2, c_emoji.shape[2]//2, 3:]=1

for j in trange(N[-1]):
    fig, axss = plt.subplots(2, 3, dpi=100, figsize=(13,8))
    c_ghost = m_ghost(c_ghost)
    c_emoji = m_emoji(c_emoji)
    for c, axs in zip([c_ghost, c_emoji], axss):
        visible = c[:,:,:,0:4].numpy()
        visible = visible.clip(0,1)
        axs[0].imshow(visible[0])
        axs[0].set_title("with predicted $\\alpha$")
        visible[:,:,:,3] = visible[:,:,:,3].round().clip(0,1)
        axs[1].set_title("with binary $\\alpha$ ($\\alpha < 0.1$ means dead)")
        axs[1].imshow(visible[0])
        axs[2].set_title("ignoring $\\alpha$")
        axs[2].imshow(visible[0,:,:,0:3])
        axs[0].set_xticks([])
        axs[1].set_xticks([])
        axs[2].set_xticks([])
        axs[0].set_yticks([])
        axs[1].set_yticks([])
        axs[2].set_yticks([])
    fig.suptitle("Growing Neural Cellular Automata", fontsize=24, y=0.97)
    plt.savefig(f'imgs/{j}.png')
    plt.close()